In [ ]:
import numpy as np
import math
import os
import matplotlib
matplotlib.rcParams["image.interpolation"] = 'none'
import matplotlib.pyplot as plt
import pathlib
import sys
import natsort
from skimage.feature import peak_local_max
from skimage import util, measure
from skimage.filters import threshold_otsu, gaussian
from skimage.segmentation import watershed
from skimage.measure import label, regionprops, regionprops_table
from skimage.morphology import closing,  disk, dilation, erosion, ball
from scipy.ndimage import distance_transform_edt
from scipy import ndimage as ndi
from skimage.transform import rescale

import napari
from tqdm import tqdm
from tifffile import imread
import tifffile

import warnings
warnings.filterwarnings("ignore")



In [4]:
#IGNORE THIS - SOMETIMES THE DECONVOLUTION RETURNS IMAGES WITH PIXEL VALUES ALL OVER THE PLACE, SO THIS CONVERTS EVERY IMAGE TO HAVE PIXEL VALUES BETWEEN 0 AND 255
def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img

In [5]:
#IGNORE THIS - EXTRACTS TIF SIZE FROM IMAGE METADATA
def pixel_size(tif_path):
    with tifffile.TiffFile(tif_path) as tif:
        tif_tags = {}
        for tag in tif.pages[0].tags.values():
            name, value = tag.name, tag.value
            tif_tags[name] = value

        x_pixel_size_um = 1/((tif_tags["XResolution"])[0]/(tif_tags["XResolution"][1]))
        y_pixel_size_um = 1/((tif_tags["YResolution"])[0]/(tif_tags["YResolution"][1]))
        try:
            z_pixel_size_um = float(str(tif_tags["IJMetadata"]).split("nscales=")[1].split(",")[2].split("\\nunit")[0])
        except:
            z_pixel_size_um = (float(str(tif_tags["ImageDescription"]).split("spacing=")[1].split("loop")[0]))
       
    original_spacing = [x_pixel_size_um,y_pixel_size_um,z_pixel_size_um]
    return original_spacing

In [ ]:
def clean_labels(path, z_size = 6.0800000, x_y_size = 0.1700744):
    image = convert(imread(path), 0,255, np.uint8)
    scale_change = z_size / x_y_size #how much bigger the z step is than x,y
    scale_value = 0.5
    new_pixel_size = (1/scale_value) * x_y_size

    #########CREATE LABELS
    rescaled_image = rescale(scale = (1,scale_value*scale_change, scale_value, scale_value), image=(image ), anti_aliasing = False)
    rescaled_image = gaussian(rescaled_image, sigma = 2)
    thresh = threshold_otsu(rescaled_image)
    rescaled_image = rescaled_image > thresh

    all_output_labels = []
    for t in range(rescaled_image.shape[0]):
        single_time = rescaled_image[t,:,:]
        distances = ndi.distance_transform_edt(erosion(single_time, ball(3)))
        coordinates = peak_local_max(distances, min_distance = int(20)) ####seed points at least 4um apart
        marker_locations = coordinates.data
        markers = np.zeros(single_time.shape, dtype=np.uint32)
        marker_indices = tuple(np.round(marker_locations).astype(int).T)
        markers[marker_indices] = np.arange(len(marker_locations)) + 1
        markers_big = dilation(markers, ball(2))
        segmented = watershed(-distances, markers_big, mask=single_time)
        table = regionprops_table(segmented, properties=('label', 'area'),)
        volume_threshold = (4/3)*np.pi * (4/new_pixel_size)**2     # radius 2um nucleus
        condition = (table['area'] >= volume_threshold)
        input_labels = table['label']
        output_labels = input_labels * condition
        output_labels = util.map_array(segmented, input_labels, output_labels)
        all_output_labels.append(output_labels)
    timelapse_labels = np.stack(all_output_labels, axis=0)
    return timelapse_labels

In [7]:
def clean_images(path, z_size = 6.0800000, x_y_size = 0.1700744):
    all_times_dapi = imread(path)[:,:,0,:,:]
    image = convert(all_times_dapi, 0,255, np.uint8)
    print(image.shape)
    scale_change = scale_change = z_size / x_y_size #how much bigger the z step is than x,y
    scale_value = 0.5
    new_pixel_size = (1/scale_value) * x_y_size

    #########CREATE LABELS
    rescaled_image = rescale(scale = (1, scale_value*scale_change, scale_value, scale_value), image=(image ), anti_aliasing = False)
    return rescaled_image

In [27]:
binary_root = "C:\\Users\\itayl\\Downloads\\test_delete\\out\\combined" # folder containing combined top+bottom binary segmentations
binary_paths = list(pathlib.Path(binary_root).rglob("*.{}".format("tif")))
combined_binary_root = "C:\\Users\\itayl\\Downloads\\test_delete\\out\\stacked"
combined_label_root = "C:\\Users\\itayl\\Downloads\\test_delete\\out\\unlinked_labels"
print(len(binary_paths))

44


In [14]:
unique_filenames = []
for path in binary_paths:
    filename = "_".join(os.path.basename(path).lower().split("_")[2:])
    if filename not in unique_filenames:
        unique_filenames.append(filename)
len(unique_filenames)

2

In [ ]:
all_images = []
for filename in unique_filenames:
    z_slices = []
    for path in binary_paths:
        if filename in str(path).lower():
            z_slices.append(path)
    z_slices.sort()
    all_images.append(z_slices)
files = all_images
sorted_files = natsort.natsorted(files)
    

In [20]:
sorted_images = []
for image in all_images:
    sorted_image = natsort.natsorted(image)
    sorted_images.append(sorted_image)

In [26]:
for image in sorted_images:
    frames = []
    for file in image:
        filename = "_".join(os.path.basename(file).lower().split("_")[2:]).split(".tif")[0]

        img = imread(file)


        frames.append(img)
    timelapse_labels = np.stack(frames, axis=0)
    print(timelapse_labels.shape)
    output_path = combined_binary_root+"\\{}_combined_timelapse.tif".format(filename)
    tifffile.imwrite(output_path, timelapse_labels, imagej=True, metadata={'axes': 'TZYX'})


(22, 11, 442, 456)
(22, 12, 409, 409)


In [31]:
combined_binary_images = list(pathlib.Path(combined_binary_root).rglob("*.{}".format("tif")))
for image in combined_binary_images:
    filename = "_".join(os.path.basename(image).lower().split("_")[2:]).replace(".tif", "")
    output_path = combined_label_root+"\\{}_FILTEREDLABELS.tif".format(filename)
    print(output_path)
    unlinked_labels = clean_labels(image)


    tifffile.imwrite(output_path, unlinked_labels.astype(np.float32), imagej=True, metadata={'axes': 'TZYX'})

C:\Users\itayl\Downloads\test_delete\out\unlinked_labels\24h-01-scene-04-p3_cropped_stiff_combined_timelapse_FILTEREDLABELS.tif
C:\Users\itayl\Downloads\test_delete\out\unlinked_labels\24h-01-scene-07-p9_cropped_stiff_combined_timelapse_FILTEREDLABELS.tif


In [ ]:
viewer = napari.Viewer()
viewer.add_labels(unlinked_labels)

<Labels layer 'labels' at 0x28fde1d3c10>